# Runke Kutta with Linear Programming

The goal of ths spreadsheed is too test methods to determin the b-Coefficients of the Runge Kutta in every step. The solution of every step should comply with $y \geq 0$
As second condition they should comply with the order conditions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg

from numba import jit, float64, stencil


rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()

from OrderCondition import *
from RKimple import *

## Set up Order Conditions

The Order Conditions are an linear equations system and can be writen as $O b = r$. 

The vector $b \in R^s$ contains the b coefficents. $s$ is the number of stages. $O$ is an $n \times s$ matrix, where $g$ is the number of Order Conditions. $r$ is an vector containing the right hand side of the Order Conditions

Has Solution ->
No contradiction -> can be avoided by choosing right $A$ matrix (and $c$) 



In order to get an optimisation problem the equation system has to be underdetermined.

This implies that $rank{O} \leq s$. 

Quadrature Conditions are linear independant
If so: $p \leq s$









In [ ]:
#Test Function
rkm = ssp104
O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)

print(O)


In [ ]:
#Are O vectors linear dependand? -> Rank of O
print ('Size of O:')
print (O.shape)

print('Rank')
print(linalg.matrix_rank(O))

Note: Order is smaler than minimum dimention of Matrix -> some way to simplify the Linear Equation System

## Positifity Constraint  

For solving an PDE with the method of lines an spartial grid  with $p$ points is introduced. 
From this we get an ODE with an $u \in R^p$.

To enshure that the solution is positive 

$ u_i^{n+1} \geq 0   \;   \forall_{i \in \{0,p-1 \}}  $

$ u_i^n + h \sum_{j=0}^{s-1} b_j h_i^j  \geq 0   \;   \forall_{i \in \{0,p-1 \}}  $


has to be fulfilled.
This infers $p$ positivity constraints to the optimisation problem. These can be written as

$u_i + h K \bullet b \geq 0$     (where $K = \big[k^1 , \cdots k^{s-1}\big] $)  



## Objective Function

Different Choices of objective function. Ideas:

$\bullet$ Penalty on large b's 

$\bullet$ Some kind of linear estimation of error term


Here we try the first approach. The Objective Function should be

$|b-rkm.b|_1$

This means that the goal is to minmise the sum of the absolute differences of the usual b vector. This is archieved by the use of slack variables

$x_1 = t_1-t_1'$ with $t_1 \geq 0$ and $ t_1 \geq 0$ 

$min(|x|_1)$ gets so $min(\mathbf{1}^T t + \mathbf{1}^T t) = min(\mathbf{1}^T \bigl(\begin{smallmatrix}
t \\ t'
\end{smallmatrix} \bigr) )$ 



## Impelmentation of RK method 

Setup: Calculate Order Condition Matrix and right hand side vector
 

a) calculate Intermediate Steps for an Runge Kutta method with arbitrary f(t,u) A and c

b) Pack the calculated Data into a Linear optimisation Problem 

   $\bullet$ Constraints:
   
   Order Condition: $Ob=r$
   
   Positifity: $u_i + h K \bullet b \geq 0$

   $\bullet$ objective function: L1 of b
   
   
    


## Testproblem

Atmospheric pollution model from Hundsdorfer & Verwer



## Testproblem Time integrator

In [ ]:
def f_sin(t,u):
    return -np.sin(t)

def f_cos(t,u):
    return np.cos(t)

def f_const(t,u):
    return 1

u0 =np.array([2.0])

t,u,b = RK_variable_b(ssp104,1,f_sin,w0=u0,t_final=100,b_fixed=False)


In [ ]:
plt.plot(t,u[0,:])

In [ ]:
plt.plot(t[1:],b.T[1:,:])

## Testproblem Harmonic Oscilator

In [ ]:
def f_A(t,u):
    A = np.array([[0,-1],[1,0]])
    c = np.array([1,1])
    #print(u)
    return A@(u-c)

u0 =np.array([1.,0.])

t,u,b = RK_variable_b(ssp104,5,f_A,w0=u0,t_final=200,b_fixed=False)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])

In [ ]:
plt.plot(t[1:],b.T[1:,:])

## Testproblem from Kopecz and Meister 2018

In [ ]:
def f_lin_I(t,u):
    a = 5 # a>0
    A = np.array([[-a,1],[a,-1]])
    return A@u

u0 =np.array([0.9,0.1])

t,u,b = RK_variable_b(ssp104,2,f_lin_I,w0=u0,t_final=100,b_fixed=False)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])

In [ ]:
plt.plot(t[1:],b.T[1:,:])

In [ ]:
ssp104.real_stability_interval()

In [ ]:
#Nonlinear test problem
def f_nonlin(t,u):
    a = 0.3
    du = np.zeros(3)
    du[0] = -(u[0]*u[1])/(u[0]+1)
    du[1] = (u[0]*u[1])/(u[0]+1) -a*u[1]
    du[2] = a*u[1]
    return du

u0 = np.array([9.98,0.01,0.01])

t,u,b = RK_variable_b(ssp104,3.8,f_nonlin,w0=u0,t_final=30,b_fixed=False)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])
plt.plot(t,u[2,:])

plt.grid()

In [ ]:
t

In [ ]:
# Brusselator test problem
def f_brusselator(t,u):
    k1 =1
    k2 =1
    k3 =1
    k4 =1
    k5 =1
    k6 =1
    du = np.zeros(6)
    du[0] = -k1*u[0]
    du[1] = -k2*u[1]*u[4]
    du[2] = k3*u[1]*u[4]
    du[3] = k4*u[4]
    du[4] = k1*u[0] - k2*u[1]*u[4] + k3*u[4]**2*u[5] - k4*u[4]
    du[5] = k2*u[1]*u[4] - k3*u[4]**2*u[5]
    return du

u0 = np.array([10.,10.,0.,0.,0.1,0.1])


t,u,b = RK_variable_b(ssp104,0.95,f_brusselator,w0=u0,t_final=6,b_fixed=False)

In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])
plt.plot(t,u[2,:])
plt.plot(t,u[3,:])
plt.plot(t,u[4,:])
plt.plot(t,u[5,:])

In [ ]:
#Robertson test problem, stiff
def f_robertson(t,u):
    a = 0.3
    du = np.zeros(3)
    du[0] = 1e4 *u[1]*u[2] - 0.04*u[0]
    du[1] = 0.04 *u[0] - 1e4*u[1]*u[2] - 3e7*u[1]**2
    du[2] = 3e7*u[1]**2
    return du

u0 = np.array([1.,0.,0.])

#t,u,b = RK_variable_b(ssp104,1e-6,f_robertson,w0=u0,t_final=1,b_fixed=True)

# Decay with Production

ODE of form $u'(t) = - \lambda u(t) + f(t)$ with $f(t) \geq 0 $ 

In [ ]:
def production(t):
    return 0.05*(1+np.sin(10*t))

def f_decay(t,u):
    l = 1
    return -l*u+production(t)

u0 = np.array([1.])

t,u,b = RK_variable_b(ssp104,10,f_decay,w0=u0,t_final=200,b_fixed=False)

In [ ]:
plt.plot(t,u[0,:])
plt.grid()

In [ ]:
plt.plot(t[1:],b.T[1:,:])

# Testproblem

In [ ]:
def f_test(t,u):
    return np.sin(10*t)*u*(1-u)

u0 = np.array([0.1])

t,u,b = RK_variable_b(ssp104,2,f_test,w0=u0,t_final=200,b_fixed=False)

In [ ]:
plt.plot(t,u[0,:])
plt.grid()

# Advection

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]

def f_centered_advection(t,u):
    du = np.zeros_like(u)
    du[1:-1] = (u[2:]-u[:-2])
    du[0] = u[1]-u[-1]
    du[-1] = u[0]-u[-2]
    return -du/(2*dx)

def f_upwind_advection(t,u):
    du = np.zeros_like(u)
    du[1:] = (u[1:]-u[:-1])
    du[0] = u[0]-u[-1]
    return -du/(dx)

#u0 = (x<0.5)*1.
#u0 = (x<0.5)+0.2
u0 = (x<0.5)+0.
#u0 = np.sin(2*np.pi*x)+1.

dt = 2.5*dx

t,u,b = RK_variable_b(ssp104,dt,f_upwind_advection,w0=u0,t_final=1,b_fixed=False)

In [ ]:
plt.pcolor(u[:,::1])
plt.colorbar()

In [ ]:
plt.plot(u[:,-1])

In [ ]:
plt.plot(t[1:],b.T[1:,:])

In most cases only very few steps require the use of the optimisation. Searching for other testproblems

## Variable coefficient advection

$u_t + (a(x,t)u)_x = 0$

$u_t = - (a(x,t)u)_x$

$\frac{\mathrm d}{\mathrm d x} \big( a(x,t)u(x,t) \big) = \frac{\mathrm d}{\mathrm d x} \big( u(x,t) \big) a(x,t) =\frac{\mathrm d}{\mathrm d x} \big( a(x,t) \big) u(x,t)$


Hom. Dirichelt Boundary at $x=0$ open Boundary on $x=1$

In [ ]:
N =20
x = np.linspace(0,1,num = N)
dx=1/(N-1)
u0 = np.sin(x)
u0 = np.zeros_like(x)
u0[7] = 1
#u0[14] = 1

def f_var_coeff_adv(t,u):
    
    #Old implementation, does not enshure conservation
    #A_upwind_advection = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))
    #a = lambda x,t: np.cos(20*x + 45*t)**2
    #a_ = lambda x,t: 2*np.cos(20*x + 45*t)*-np.sin(20*x + 45*t)*20
    #return np.diag(a(x,t))@A_upwind_advection@u +  a_(x,t)*u

    A_upwind_advection = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))
    a = lambda x,t: np.cos(20*x + 45*t)**2
    return A_upwind_advection@(a(x,t)*u)


dt = 0.4*4*dx

t,u,b = RK_variable_b(rk4,dt,f_var_coeff_adv,w0=u0,t_final=20,b_fixed=True)

In [ ]:
plt.pcolor(u[:,::1])
plt.colorbar()

In [ ]:
plt.pcolor(u[:,::1]<0)